## 3.1.3 Linear Regression Example with Word2Vec

### Word2Vec Feature Example

In [1]:
import os
import re

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
DATA_IN_PATH = './data_in/'
TRAIN_CLEAN_DATA = 'train_clean.csv'

RANDOM_SEED = 42
TEST_SPLIT = 0.2

In [3]:
train_data = pd.read_csv(DATA_IN_PATH + TRAIN_CLEAN_DATA)

In [4]:
reviews = list(train_data['review'])
sentiments = list(train_data['sentiment'])

In [5]:
sentences = []
for review in reviews:
    sentences.append(review.split())

In [6]:
num_features = 300    
min_word_count = 40   
num_workers = 4       
context = 10          
downsampling = 1e-3 

In [7]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
   level=logging.INFO)

In [8]:
from gensim.models import word2vec

model = word2vec.Word2Vec(sentences, workers=num_workers, \
           size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

2021-03-18 16:05:04,071 : INFO : collecting all words and their counts
2021-03-18 16:05:04,072 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-03-18 16:05:04,249 : INFO : PROGRESS: at sentence #10000, processed 1205223 words, keeping 51374 word types
2021-03-18 16:05:04,452 : INFO : PROGRESS: at sentence #20000, processed 2396605 words, keeping 67660 word types
2021-03-18 16:05:04,543 : INFO : collected 74065 word types from a corpus of 2988089 raw words and 25000 sentences
2021-03-18 16:05:04,544 : INFO : Loading a fresh vocabulary
2021-03-18 16:05:04,576 : INFO : effective_min_count=40 retains 8160 unique words (11% of original 74065, drops 65905)
2021-03-18 16:05:04,577 : INFO : effective_min_count=40 leaves 2627273 word corpus (87% of original 2988089, drops 360816)
2021-03-18 16:05:04,597 : INFO : deleting the raw counts dictionary of 74065 items
2021-03-18 16:05:04,599 : INFO : sample=0.001 downsamples 30 most-common words
2021-03-18 16:05:04,600 :

In [9]:
def get_features(words, model, num_features):
    feature_vector = np.zeros((num_features),dtype=np.float32)

    num_words = 0
    index2word_set = set(model.wv.index2word)

    for w in words:
        if w in index2word_set:
            num_words += 1
            feature_vector = np.add(feature_vector, model[w])

    feature_vector = np.divide(feature_vector, num_words)
    return feature_vector

In [10]:
def get_dataset(reviews, model, num_features):
    dataset = list()

    for s in reviews:
        dataset.append(get_features(s, model, num_features))

    reviewFeatureVecs = np.stack(dataset)
    
    return reviewFeatureVecs

In [11]:
test_data_vecs = get_dataset(sentences, model, num_features)

<ipython-input-9-48cdc3edcd51>:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  feature_vector = np.add(feature_vector, model[w])


In [12]:
from sklearn.model_selection import train_test_split
import numpy as np

X = test_data_vecs
y = np.array(sentiments)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SPLIT, random_state=RANDOM_SEED)

In [13]:
from sklearn.linear_model import LogisticRegression

lgs = LogisticRegression(class_weight='balanced')
lgs.fit(X_train, y_train)

/home/dacon/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(class_weight='balanced')

In [14]:
print("Accuracy: %f" % lgs.score(X_test, y_test)) 

Accuracy: 0.862000


In [15]:
TEST_CLEAN_DATA = 'test_clean.csv'

test_data = pd.read_csv(DATA_IN_PATH + TEST_CLEAN_DATA)

test_review = list(test_data['review'])

In [16]:
test_data.head(5)

,review,id
0,naturally film main themes mortality nostalgia...,"""12311_10"""
1,movie disaster within disaster film full great...,"""8348_2"""
2,movie kids saw tonight child loved one point k...,"""5828_4"""
3,afraid dark left impression several different ...,"""7186_2"""
4,accurate depiction small time mob life filmed ...,"""12128_7"""


In [17]:
test_sentences = list()
for review in test_review:
    test_sentences.append(review.split())

In [18]:
test_data_vecs = get_dataset(test_sentences, model, num_features)

<ipython-input-9-48cdc3edcd51>:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  feature_vector = np.add(feature_vector, model[w])


In [19]:
DATA_OUT_PATH = './data_out/'

test_predicted = lgs.predict(test_data_vecs)

if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)
    
ids = list(test_data['id'])
answer_dataset = pd.DataFrame({'id': ids, 'sentiment': test_predicted})
answer_dataset.to_csv(DATA_OUT_PATH + 'lgs_w2v_answer.csv', index=False, quoting=3)

In [20]:
model_name = "300features_40minwords_10context"
model.save(model_name)

2021-03-18 16:05:57,664 : INFO : saving Word2Vec object under 300features_40minwords_10context, separately None
2021-03-18 16:05:57,665 : INFO : not storing attribute vectors_norm
2021-03-18 16:05:57,666 : INFO : not storing attribute cum_table
2021-03-18 16:05:57,857 : INFO : saved 300features_40minwords_10context
